In [2]:
import pandas as pd
import pdfplumber
import os

# ==============================================================================
# ETAPA 1: FUNÇÃO PARA LIMPAR A AUDIÊNCIA BRUTA
# ==============================================================================

def limpar_audiencia_bruta(caminho_arquivo_bruto):
    """
    Lê o arquivo de audiência bruto, remove as 6 primeiras e a última linha,
    seleciona, renomeia as colunas e retorna um DataFrame limpo.
    """
    print("Iniciando Etapa 1: Limpeza do arquivo de audiência bruto...")
    
    try:
        # Dicionário que define as colunas a serem mantidas e seus novos nomes
        colunas_para_renomear = {
            'MIN': 'HORA',
            'TV GAZETA': 'GLOBO',
            'TV VITORIA': 'RECORD',
            'TV SIM': 'SBT',
            'CONTEÚDO TV/VÍDEO S/ REFERÊNCIA': 'Conteúdo de TV/Vídeo sem referência',
            'TOTAL LIGADOS': 'Total Ligados Especial'
        }

        # Lê o arquivo original, pulando as 6 primeiras linhas
        # O skipfooter exige a engine 'python'
        df = pd.read_csv(caminho_arquivo_bruto, sep=';', skiprows=6, skipfooter=1, engine='python')

        # Pega a lista das colunas originais que queremos manter
        colunas_originais = list(colunas_para_renomear.keys())

        # Filtra o DataFrame, mantendo apenas as colunas desejadas
        df_selecionado = df[colunas_originais]

        # Renomeia as colunas e cria o DataFrame final
        df_limpo = df_selecionado.rename(columns=colunas_para_renomear)

        print("✅ Etapa 1 concluída: Audiência bruta limpa e formatada em memória.")
        return df_limpo

    except FileNotFoundError:
        print(f"❌ ERRO na Etapa 1: O arquivo de entrada '{caminho_arquivo_bruto}' não foi encontrado.")
        return None
    except KeyError as e:
        print(f"❌ ERRO na Etapa 1: Uma ou mais colunas para renomear não foram encontradas: {e}")
        return None
    except Exception as e:
        print(f"❌ Ocorreu um erro inesperado na Etapa 1: {e}")
        return None

# ==============================================================================
# ETAPA 2: FUNÇÃO PARA PROCESSAR O RUNDOWN (PDF)
# ==============================================================================

def processar_rundown_pdf(caminho_pdf):
    """
    Função estável para extrair os dados do rundown (espelho) de um PDF.
    """
    print("\nIniciando Etapa 2: Processamento do Rundown (PDF)...")
    if not os.path.exists(caminho_pdf):
        print(f"❌ ERRO na Etapa 2: O arquivo PDF não foi encontrado em '{caminho_pdf}'")
        return None

    dados_finais = []
    bloco_numero = 1
    with pdfplumber.open(caminho_pdf) as pdf:
        for pagina in pdf.pages:
            tabelas = pagina.extract_tables()
            for tabela in tabelas:
                for linha in tabela:
                    if linha and linha[0] and isinstance(linha[0], str) and 'Break' in linha[0]:
                        bloco_numero += 1
                        continue
                    if len(linha) >= 10 and linha[-1] and isinstance(linha[-1], str) and linha[-1].isdigit():
                        dados_finais.append({
                            'Bloco': f"Bloco {bloco_numero:02d}", 'Tipo': linha[1], 'Retranca': linha[2],
                            'Início (Lauda)': linha[6], 'Fim (Lauda)': linha[7], 'Ordem': int(linha[9])
                        })
    if not dados_finais:
        print("⚠️ Atenção na Etapa 2: Nenhum dado de lauda válido foi extraído do PDF.")
        return None
    df = pd.DataFrame(dados_finais).sort_values(by='Ordem').reset_index(drop=True)
    print("✅ Etapa 2 concluída: Rundown (PDF) processado com sucesso.")
    return df

# ==============================================================================
# ETAPA 3: FUNÇÃO PARA FAZER O MERGE FINAL
# ==============================================================================

def fazer_merge_final(df_espelho_orig, df_audiencia_limpa):
    """
    Recebe os DataFrames do espelho e da audiência limpa,
    e realiza a expansão e o merge em nível de segundo.
    """
    print("\nIniciando Etapa 3: Expansão e Merge dos dados...")
    
    # --- 3.1: Processando e expandindo audiência ---
    df_minamin = df_audiencia_limpa.copy()
    df_minamin['HORA_base'] = pd.to_datetime(df_minamin['HORA'], format='%H:%M', errors='coerce').dt.time
    df_minamin.dropna(subset=['HORA_base'], inplace=True)
    if df_minamin.empty:
        print("❌ ERRO na Etapa 3: O dataframe de audiência ficou vazio após a limpeza. Verifique o formato da hora no arquivo bruto.")
        return None
    df_minamin_segundos = df_minamin.copy()
    df_minamin_segundos['datetime_segundo_audiencia'] = df_minamin_segundos['HORA_base'].apply(
        lambda t: pd.date_range(start=f"1900-01-01 {t}", periods=60, freq='S')
    )
    df_minamin_segundos = df_minamin_segundos.explode('datetime_segundo_audiencia')
    
    # --- 3.2: Processando e expandindo espelho ---
    df_espelho = df_espelho_orig.copy()
    df_espelho.rename(columns={'Início (Lauda)': 'inicio_lauda', 'Fim (Lauda)': 'fim_lauda'}, inplace=True)
    df_espelho['inicio_lauda_dt'] = pd.to_datetime('1900-01-01 ' + df_espelho['inicio_lauda'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    df_espelho['fim_lauda_dt'] = pd.to_datetime('1900-01-01 ' + df_espelho['fim_lauda'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    df_espelho.dropna(subset=['inicio_lauda_dt', 'fim_lauda_dt'], inplace=True)
    df_espelho = df_espelho[df_espelho['fim_lauda_dt'] >= df_espelho['inicio_lauda_dt']].copy()
    if df_espelho.empty:
        print("⚠️ Atenção na Etapa 3: O dataframe do espelho ficou vazio após a limpeza.")
        return None
    df_espelho['datetime_segundo_espelho'] = df_espelho.apply(
        lambda row: pd.date_range(start=row['inicio_lauda_dt'], end=row['fim_lauda_dt'], freq='S'), axis=1
    )
    df_espelho_segundos = df_espelho.explode('datetime_segundo_espelho').drop_duplicates(subset=['datetime_segundo_espelho'], keep='first')

    # --- 3.3: Merge Final ---
    print("Realizando o merge dos dados...")
    df_merged_segundos = pd.merge(
        df_minamin_segundos, df_espelho_segundos,
        left_on='datetime_segundo_audiencia', right_on='datetime_segundo_espelho', how='left'
    )

    # --- 3.4: Limpeza e Salvamento ---
    colunas_para_remover = ['HORA_base', 'datetime_segundo_espelho', 'inicio_lauda_dt', 'fim_lauda_dt']
    df_merged_segundos.drop(columns=colunas_para_remover, inplace=True, errors='ignore')
    
    print("✅ Etapa 3 concluída: Merge finalizado com sucesso.")
    return df_merged_segundos

# ==============================================================================
# PONTO DE PARTIDA PRINCIPAL DO SCRIPT
# ==============================================================================

if __name__ == "__main__":
    # --- Defina os NOMES DOS ARQUIVOS DE ENTRADA aqui ---
    ARQUIVO_AUDIENCIA_BRUTO = "Realtime_bruto.csv"
    ARQUIVO_RUNDOWN_PDF = "rundown-display.pdf"
    
    # --- Defina o NOME DO ARQUIVO DE SAÍDA aqui ---
    ARQUIVO_SAIDA_FINAL = "audiencia_espelho_merged_segundos.csv"

    print("======================================================")
    print("🚀 INICIANDO PIPELINE COMPLETO DE DADOS 🚀")
    print("======================================================")
    
    # Roda a Etapa 1
    df_audiencia_limpa = limpar_audiencia_bruta(ARQUIVO_AUDIENCIA_BRUTO)
    
    # Roda a Etapa 2
    df_espelho_gerado = processar_rundown_pdf(ARQUIVO_RUNDOWN_PDF)
    
    # Roda a Etapa 3 apenas se as etapas anteriores foram bem-sucedidas
    if df_audiencia_limpa is not None and df_espelho_gerado is not None:
        df_final = fazer_merge_final(df_espelho_gerado, df_audiencia_limpa)
        
        if df_final is not None and not df_final.empty:
            # Salva o resultado final
            df_final.to_csv(ARQUIVO_SAIDA_FINAL, index=False, encoding='utf-8-sig')
            print("\n======================================================")
            print(f"🏆 PIPELINE FINALIZADO COM SUCESSO! 🏆")
            print(f"Arquivo final salvo em: '{ARQUIVO_SAIDA_FINAL}'")
            print("======================================================")
            print("\nVisualização das 5 primeiras linhas do resultado:")
            print(df_final.head())
        else:
            print("\nO pipeline foi interrompido na Etapa 3 porque o resultado do merge foi vazio.")
    else:
        print("\nO pipeline foi interrompido porque uma das etapas iniciais (1 ou 2) falhou.")

🚀 INICIANDO PIPELINE COMPLETO DE DADOS 🚀
Iniciando Etapa 1: Limpeza do arquivo de audiência bruto...
✅ Etapa 1 concluída: Audiência bruta limpa e formatada em memória.

Iniciando Etapa 2: Processamento do Rundown (PDF)...
✅ Etapa 2 concluída: Rundown (PDF) processado com sucesso.

Iniciando Etapa 3: Expansão e Merge dos dados...
Realizando o merge dos dados...
✅ Etapa 3 concluída: Merge finalizado com sucesso.

🏆 PIPELINE FINALIZADO COM SUCESSO! 🏆
Arquivo final salvo em: 'audiencia_espelho_merged_segundos.csv'

Visualização das 5 primeiras linhas do resultado:
    HORA GLOBO RECORD   SBT Conteúdo de TV/Vídeo sem referência  \
0  08:40  7,72   3,83  1,20                                8,23   
1  08:40  7,72   3,83  1,20                                8,23   
2  08:40  7,72   3,83  1,20                                8,23   
3  08:40  7,72   3,83  1,20                                8,23   
4  08:40  7,72   3,83  1,20                                8,23   

  Total Ligados Especial datet

C:\Temp\ipykernel_26344\1534761766.py:108: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  lambda t: pd.date_range(start=f"1900-01-01 {t}", periods=60, freq='S')
C:\Temp\ipykernel_26344\1534761766.py:123: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  lambda row: pd.date_range(start=row['inicio_lauda_dt'], end=row['fim_lauda_dt'], freq='S'), axis=1
